<a href="https://colab.research.google.com/github/lehai0609/KagglePlayground/blob/main/Playground_S5E7_Improving_0_973279.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 0. Overview

In this notebook, I'm doing EDA for the dataset and perform a baseline model based on EDA analysis.
It's a very standardized process, just like the dataset provided in this month's challenge

In [1]:
# prompt: Read train.csv & test.csv from folder: /content/drive/MyDrive/Kaggle/Playground S5E7

import pandas as pd

train_df = pd.read_csv('/content/drive/MyDrive/Kaggle/Playground S5E7/train.csv')
test_df = pd.read_csv('/content/drive/MyDrive/Kaggle/Playground S5E7/test.csv')

In [2]:
train_df.head()

,id,Time_spent_Alone,Stage_fear,Social_event_attendance,Going_outside,Drained_after_socializing,Friends_circle_size,Post_frequency,Personality
0,0,0.0,No,6.0,4.0,No,15.0,5.0,Extrovert
1,1,1.0,No,7.0,3.0,No,10.0,8.0,Extrovert
2,2,6.0,Yes,1.0,0.0,NaN,3.0,0.0,Introvert
3,3,3.0,No,7.0,3.0,No,11.0,5.0,Extrovert
4,4,1.0,No,4.0,4.0,No,13.0,NaN,Extrovert


In [3]:
test_df.head()

,id,Time_spent_Alone,Stage_fear,Social_event_attendance,Going_outside,Drained_after_socializing,Friends_circle_size,Post_frequency
0,18524,3.0,No,7.0,4.0,No,6.0,NaN
1,18525,NaN,Yes,0.0,0.0,Yes,5.0,1.0
2,18526,3.0,No,5.0,6.0,No,15.0,9.0
3,18527,3.0,No,4.0,4.0,No,5.0,6.0
4,18528,9.0,Yes,1.0,2.0,Yes,1.0,1.0


# III. TARGET ENCODED APPROACH

In [4]:
# Verify GPU availability (after restart)
import cupy as cp
import cudf as cd

# Check GPU info
device = cp.cuda.Device()
print(f"GPU ID: {device.id}")
print(f"GPU Memory: {device.mem_info[1] / 1024**3:.1f} GB total")
print(f"GPU Memory Available: {device.mem_info[0] / 1024**3:.1f} GB free")

# Test cuDF
try:
    test_cudf = cd.DataFrame({'test': [1, 2, 3]})
    print("✅ cuDF working correctly")
    print(f"cuDF version: {cd.__version__}")
except Exception as e:
    print(f"❌ cuDF error: {e}")

# Test cuML
try:
    from cuml.preprocessing import LabelEncoder
    print("✅ cuML working correctly")
except Exception as e:
    print(f"❌ cuML error: {e}")

# Test CuPy
try:
    test_array = cp.array([1, 2, 3])
    print("✅ CuPy working correctly")
    print(f"CuPy version: {cp.__version__}")
except Exception as e:
    print(f"❌ CuPy error: {e}")

GPU ID: 0
GPU Memory: 14.7 GB total
GPU Memory Available: 14.6 GB free
✅ cuDF working correctly
cuDF version: 25.02.01
✅ cuML working correctly
✅ CuPy working correctly
CuPy version: 13.3.0


In [5]:
import cudf as cd
import cupy as cp
import pandas as pd
import numpy as np
from cuml.model_selection import train_test_split
from cuml.preprocessing import LabelEncoder as cuLabelEncoder
from cuml.preprocessing import SimpleImputer as cuSimpleImputer
from cuml.metrics import roc_auc_score as cu_roc_auc_score
from cuml.ensemble import RandomForestClassifier as cuRandomForestClassifier
import xgboost as xgb
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import roc_auc_score, f1_score, accuracy_score
import warnings
warnings.filterwarnings('ignore')

# Check GPU availability
print(f"GPU memory: {cp.cuda.Device().mem_info}")
print("RAPIDS cuDF and cuML loaded successfully")

# Define constants
NUMERICAL_FEATURES = ['Time_spent_Alone', 'Social_event_attendance', 'Going_outside', 'Friends_circle_size', 'Post_frequency']
CATEGORICAL_FEATURES = ['Stage_fear', 'Drained_after_socializing']
FEATURES = NUMERICAL_FEATURES + CATEGORICAL_FEATURES
STATS = ["mean", "count", "std"]
STATS2 = ["mean", "count"]
FOLDS = 5

print("Constants defined successfully")

GPU memory: (15716188160, 15828320256)
RAPIDS cuDF and cuML loaded successfully
Constants defined successfully


In [6]:
test_df.head()

,id,Time_spent_Alone,Stage_fear,Social_event_attendance,Going_outside,Drained_after_socializing,Friends_circle_size,Post_frequency
0,18524,3.0,No,7.0,4.0,No,6.0,NaN
1,18525,NaN,Yes,0.0,0.0,Yes,5.0,1.0
2,18526,3.0,No,5.0,6.0,No,15.0,9.0
3,18527,3.0,No,4.0,4.0,No,5.0,6.0
4,18528,9.0,Yes,1.0,2.0,Yes,1.0,1.0


In [7]:
# 3-Level Stacking with Target Encoding and RAPIDS GPU Acceleration
import cudf as cd
import cupy as cp
import pandas as pd
import numpy as np
from cuml.preprocessing import LabelEncoder as cuLabelEncoder
from cuml.ensemble import RandomForestClassifier as cuRandomForestClassifier
from cuml.linear_model import LogisticRegression as cuLogisticRegression
import xgboost as xgb
import lightgbm as lgb
from sklearn.model_selection import StratifiedKFold
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import roc_auc_score, f1_score, accuracy_score
import warnings
warnings.filterwarnings('ignore')

# Constants
NUMERICAL_FEATURES = ['Time_spent_Alone', 'Social_event_attendance', 'Going_outside', 'Friends_circle_size', 'Post_frequency']
CATEGORICAL_FEATURES = ['Stage_fear', 'Drained_after_socializing']
FEATURES = NUMERICAL_FEATURES + CATEGORICAL_FEATURES
STATS = ["mean", "count", "std"]
STATS2 = ["mean", "count"]
FOLDS = 5

# Load and prepare data
train_df = pd.read_csv('/content/drive/MyDrive/Kaggle/Playground S5E7/train.csv')
test_df = pd.read_csv('/content/drive/MyDrive/Kaggle/Playground S5E7/test.csv')

# Fill nulls in numerical features
for col in NUMERICAL_FEATURES:
    train_df[col] = train_df[col].fillna(-1)
    test_df[col] = test_df[col].fillna(-1)

train_df['Personality_binary'] = (train_df['Personality'] == 'Introvert').astype(int)

# Initialize CV and storage
kf = StratifiedKFold(n_splits=FOLDS, shuffle=True, random_state=42)
kf2 = StratifiedKFold(n_splits=FOLDS, shuffle=True, random_state=43)

# Level 1 predictions storage
level1_train = np.zeros((len(train_df), 4))  # 4 base models
level1_test = np.zeros((len(test_df), 4))    # 4 base models
level2_train = np.zeros(len(train_df))       # Meta-learner
level2_test = np.zeros(len(test_df))         # Meta-learner

print("Starting 3-Level Stacking with Target Encoding...")

def create_target_encoded_features(X_train, X_valid, X_test, y_train):
    """Create the same 66 target-encoded features as original notebook"""

    # Handle missing values
    num_imputer = SimpleImputer(strategy='median')
    X_train[NUMERICAL_FEATURES] = num_imputer.fit_transform(X_train[NUMERICAL_FEATURES])
    X_valid[NUMERICAL_FEATURES] = num_imputer.transform(X_valid[NUMERICAL_FEATURES])
    X_test[NUMERICAL_FEATURES] = num_imputer.transform(X_test[NUMERICAL_FEATURES])

    cat_imputer = SimpleImputer(strategy='most_frequent')
    X_train[CATEGORICAL_FEATURES] = cat_imputer.fit_transform(X_train[CATEGORICAL_FEATURES])
    X_valid[CATEGORICAL_FEATURES] = cat_imputer.transform(X_valid[CATEGORICAL_FEATURES])
    X_test[CATEGORICAL_FEATURES] = cat_imputer.transform(X_test[CATEGORICAL_FEATURES])

    # Initialize feature storage
    train_features = X_train[FEATURES].copy()
    valid_features = X_valid.copy()
    test_features = X_test.copy()

    # Inner loop for target encoding (prevent leakage)
    te1_features, te2_features = {}, {}

    for inner_fold, (train_idx2, valid_idx2) in enumerate(kf2.split(X_train, y_train)):
        X_train2 = X_train.iloc[train_idx2].copy()
        X_valid2 = X_train.iloc[valid_idx2].copy()

        # TE1: Individual numerical target encoding
        for col in NUMERICAL_FEATURES:
            X_train2[f'{col}_binned'] = pd.qcut(X_train2[col], q=10, duplicates='drop')
            grouped_stats = X_train2.groupby(f'{col}_binned')['Personality_binary'].agg(STATS)
            grouped_stats.columns = [f"TE1_{col}_{s}" for s in STATS]

            global_mean = X_train2['Personality_binary'].mean()
            grouped_stats[f'TE1_{col}_smoothed'] = (
                (grouped_stats[f'TE1_{col}_mean'] * grouped_stats[f'TE1_{col}_count'] + global_mean * 10) /
                (grouped_stats[f'TE1_{col}_count'] + 10)
            )

            X_valid2[f'{col}_binned'] = pd.cut(X_valid2[col],
                bins=X_train2[f'{col}_binned'].cat.categories.left.tolist() + [X_train2[f'{col}_binned'].cat.categories.right[-1]])

            for stat in STATS + ['smoothed']:
                col_name = f'TE1_{col}_{stat}'
                mapped_values = X_valid2[f'{col}_binned'].map(grouped_stats[col_name])
                X_valid2[col_name] = pd.Series(mapped_values, dtype='float64').fillna(global_mean)

                if col_name not in te1_features:
                    te1_features[col_name] = np.zeros(len(X_train))
                te1_features[col_name][valid_idx2] = X_valid2[col_name].values

        # TE2: Interaction target encoding
        interactions = [
            ('Social_event_attendance', 'Time_spent_Alone'),
            ('Stage_fear', 'Drained_after_socializing'),
            ('Friends_circle_size', 'Going_outside')
        ]

        for feat1, feat2 in interactions:
            if feat1 in NUMERICAL_FEATURES:
                X_train2[f'{feat1}_bin'] = pd.qcut(X_train2[feat1], q=5, duplicates='drop')
                feat1_use = f'{feat1}_bin'
            else:
                feat1_use = feat1

            if feat2 in NUMERICAL_FEATURES:
                X_train2[f'{feat2}_bin'] = pd.qcut(X_train2[feat2], q=5, duplicates='drop')
                feat2_use = f'{feat2}_bin'
            else:
                feat2_use = feat2

            interaction_name = f'{feat1}_{feat2}_interaction'
            X_train2[interaction_name] = X_train2[feat1_use].astype(str) + '_' + X_train2[feat2_use].astype(str)

            grouped_stats = X_train2.groupby(interaction_name)['Personality_binary'].agg(STATS2)
            grouped_stats.columns = [f"TE2_{interaction_name}_{s}" for s in STATS2]
            grouped_stats[f'TE2_{interaction_name}_smoothed'] = (
                (grouped_stats[f'TE2_{interaction_name}_mean'] * grouped_stats[f'TE2_{interaction_name}_count'] + global_mean * 5) /
                (grouped_stats[f'TE2_{interaction_name}_count'] + 5)
            )

            if feat1 in NUMERICAL_FEATURES:
                X_valid2[f'{feat1}_bin'] = pd.cut(X_valid2[feat1],
                    bins=X_train2[f'{feat1}_bin'].cat.categories.left.tolist() + [X_train2[f'{feat1}_bin'].cat.categories.right[-1]])
            if feat2 in NUMERICAL_FEATURES:
                X_valid2[f'{feat2}_bin'] = pd.cut(X_valid2[feat2],
                    bins=X_train2[f'{feat2}_bin'].cat.categories.left.tolist() + [X_train2[f'{feat2}_bin'].cat.categories.right[-1]])

            X_valid2[interaction_name] = X_valid2[feat1_use].astype(str) + '_' + X_valid2[feat2_use].astype(str)

            for stat in STATS2 + ['smoothed']:
                col_name = f'TE2_{interaction_name}_{stat}'
                mapped_values = X_valid2[interaction_name].map(grouped_stats[col_name])
                X_valid2[col_name] = pd.Series(mapped_values, dtype='float64').fillna(global_mean)

                if col_name not in te2_features:
                    te2_features[col_name] = np.zeros(len(X_train))
                te2_features[col_name][valid_idx2] = X_valid2[col_name].values

    # Add inner loop features
    for col_name, values in te1_features.items():
        train_features[col_name] = values
    for col_name, values in te2_features.items():
        train_features[col_name] = values

    # Create features for validation/test using full training data
    # TE1 for validation/test
    for col in NUMERICAL_FEATURES:
        X_train[f'{col}_binned'] = pd.qcut(X_train[col], q=10, duplicates='drop')
        grouped_stats = X_train.groupby(f'{col}_binned')['Personality_binary'].agg(STATS)
        grouped_stats.columns = [f"TE1_{col}_{s}" for s in STATS]

        global_mean = X_train['Personality_binary'].mean()
        grouped_stats[f'TE1_{col}_smoothed'] = (
            (grouped_stats[f'TE1_{col}_mean'] * grouped_stats[f'TE1_{col}_count'] + global_mean * 10) /
            (grouped_stats[f'TE1_{col}_count'] + 10)
        )

        X_valid[f'{col}_binned'] = pd.cut(X_valid[col],
            bins=X_train[f'{col}_binned'].cat.categories.left.tolist() + [X_train[f'{col}_binned'].cat.categories.right[-1]])
        X_test[f'{col}_binned'] = pd.cut(X_test[col],
            bins=X_train[f'{col}_binned'].cat.categories.left.tolist() + [X_train[f'{col}_binned'].cat.categories.right[-1]])

        for stat in STATS + ['smoothed']:
            col_name = f'TE1_{col}_{stat}'
            mapped_valid = X_valid[f'{col}_binned'].map(grouped_stats[col_name])
            mapped_test = X_test[f'{col}_binned'].map(grouped_stats[col_name])
            valid_features[col_name] = pd.Series(mapped_valid, dtype='float64').fillna(global_mean)
            test_features[col_name] = pd.Series(mapped_test, dtype='float64').fillna(global_mean)

    # TE2 for validation/test
    for feat1, feat2 in interactions:
        if feat1 in NUMERICAL_FEATURES:
            X_train[f'{feat1}_bin'] = pd.qcut(X_train[feat1], q=5, duplicates='drop')
            feat1_use = f'{feat1}_bin'
        else:
            feat1_use = feat1

        if feat2 in NUMERICAL_FEATURES:
            X_train[f'{feat2}_bin'] = pd.qcut(X_train[feat2], q=5, duplicates='drop')
            feat2_use = f'{feat2}_bin'
        else:
            feat2_use = feat2

        interaction_name = f'{feat1}_{feat2}_interaction'
        X_train[interaction_name] = X_train[feat1_use].astype(str) + '_' + X_train[feat2_use].astype(str)

        grouped_stats = X_train.groupby(interaction_name)['Personality_binary'].agg(STATS2)
        grouped_stats.columns = [f"TE2_{interaction_name}_{s}" for s in STATS2]
        grouped_stats[f'TE2_{interaction_name}_smoothed'] = (
            (grouped_stats[f'TE2_{interaction_name}_mean'] * grouped_stats[f'TE2_{interaction_name}_count'] + global_mean * 5) /
            (grouped_stats[f'TE2_{interaction_name}_count'] + 5)
        )

        for dataset, features_df in [(X_valid, valid_features), (X_test, test_features)]:
            if feat1 in NUMERICAL_FEATURES:
                dataset[f'{feat1}_bin'] = pd.cut(dataset[feat1],
                    bins=X_train[f'{feat1}_bin'].cat.categories.left.tolist() + [X_train[f'{feat1}_bin'].cat.categories.right[-1]])
            if feat2 in NUMERICAL_FEATURES:
                dataset[f'{feat2}_bin'] = pd.cut(dataset[feat2],
                    bins=X_train[f'{feat2}_bin'].cat.categories.left.tolist() + [X_train[f'{feat2}_bin'].cat.categories.right[-1]])

            dataset[interaction_name] = dataset[feat1_use].astype(str) + '_' + dataset[feat2_use].astype(str)

            for stat in STATS2 + ['smoothed']:
                col_name = f'TE2_{interaction_name}_{stat}'
                mapped_values = dataset[interaction_name].map(grouped_stats[col_name])
                features_df[col_name] = pd.Series(mapped_values, dtype='float64').fillna(global_mean)

    # FE3: Non-target statistical features
    for cat_col in CATEGORICAL_FEATURES:
        for num_col in NUMERICAL_FEATURES:
            grouped = X_train.groupby(cat_col)[num_col].agg(['mean', 'std', 'median'])
            grouped.columns = [f"FE3_{cat_col}_{num_col}_{s}" for s in ['mean', 'std', 'median']]

            for stat in ['mean', 'std', 'median']:
                col_name = f"FE3_{cat_col}_{num_col}_{stat}"
                default_val = X_train[num_col].mean() if stat != 'std' else X_train[num_col].std()

                mapped_train = X_train[cat_col].map(grouped[col_name])
                mapped_valid = X_valid[cat_col].map(grouped[col_name])
                mapped_test = X_test[cat_col].map(grouped[col_name])

                train_features[col_name] = pd.Series(mapped_train, dtype='float64').fillna(default_val)
                valid_features[col_name] = pd.Series(mapped_valid, dtype='float64').fillna(default_val)
                test_features[col_name] = pd.Series(mapped_test, dtype='float64').fillna(default_val)

    # Label encode categorical features
    le_stage = LabelEncoder()
    le_drained = LabelEncoder()

    train_features['Stage_fear'] = le_stage.fit_transform(train_features['Stage_fear'])
    train_features['Drained_after_socializing'] = le_drained.fit_transform(train_features['Drained_after_socializing'])

    valid_features['Stage_fear'] = le_stage.transform(valid_features['Stage_fear'])
    valid_features['Drained_after_socializing'] = le_drained.transform(valid_features['Drained_after_socializing'])

    test_features['Stage_fear'] = le_stage.transform(test_features['Stage_fear'])
    test_features['Drained_after_socializing'] = le_drained.transform(test_features['Drained_after_socializing'])

    return train_features, valid_features, test_features

# LEVEL 1: Train base models
for fold, (train_index, valid_index) in enumerate(kf.split(train_df, train_df['Personality_binary'])):
    print(f"\n=== FOLD {fold + 1} ===")

    X_train = train_df.iloc[train_index][FEATURES + ['Personality_binary']].copy()
    X_valid = train_df.iloc[valid_index][FEATURES].copy()
    X_test = test_df[FEATURES].copy()

    y_train = X_train['Personality_binary']
    y_valid = train_df.iloc[valid_index]['Personality_binary']

    # Create 66 target-encoded features
    train_features, valid_features, test_features = create_target_encoded_features(X_train, X_valid, X_test, y_train)

    print(f"Created {train_features.shape[1]} features")

    # Ensure consistent feature order and reset index
    feature_cols = train_features.columns.tolist()
    train_features = train_features[feature_cols].reset_index(drop=True)
    valid_features = valid_features[feature_cols].reset_index(drop=True)
    test_features = test_features[feature_cols].reset_index(drop=True)

    # Convert to GPU format with proper index handling
    train_gpu = cd.from_pandas(train_features)
    valid_gpu = cd.from_pandas(valid_features)
    test_gpu = cd.from_pandas(test_features)

    y_train_gpu = cd.from_pandas(pd.Series(y_train.values))

    scale_pos_weight = len(y_train[y_train==0]) / len(y_train[y_train==1])

    # Model 1: XGBoost with GPU
    model1 = xgb.XGBClassifier(
        objective='binary:logistic', tree_method='gpu_hist', gpu_id=0,
        max_depth=6, learning_rate=0.02, n_estimators=800,
        scale_pos_weight=scale_pos_weight, random_state=42
    )
    model1.fit(train_features, y_train, eval_set=[(valid_features, y_valid)], verbose=False)

    # Model 2: LightGBM with GPU
    model2 = lgb.LGBMClassifier(
        objective='binary', device='gpu', gpu_platform_id=0, gpu_device_id=0,
        max_depth=6, learning_rate=0.02, n_estimators=800,
        scale_pos_weight=scale_pos_weight, random_state=42, verbose=-1
    )
    model2.fit(train_features, y_train, eval_set=[(valid_features, y_valid)], callbacks=[lgb.early_stopping(50)])

    # Model 3: cuML RandomForest
    model3 = cuRandomForestClassifier(n_estimators=200, max_depth=10)
    model3.fit(train_gpu, y_train_gpu)

    # Model 4: cuML LogisticRegression
    model4 = cuLogisticRegression(max_iter=1000)
    model4.fit(train_gpu, y_train_gpu)

    # Level 1 predictions
    pred1_valid = model1.predict_proba(valid_features)[:, 1]
    pred1_test = model1.predict_proba(test_features)[:, 1]

    pred2_valid = model2.predict_proba(valid_features)[:, 1]
    pred2_test = model2.predict_proba(test_features)[:, 1]

    # Handle cuML predictions properly
    pred3_proba_valid = model3.predict_proba(valid_gpu)
    pred3_valid = cp.asnumpy(pred3_proba_valid.iloc[:, 1])

    pred3_proba_test = model3.predict_proba(test_gpu)
    pred3_test = cp.asnumpy(pred3_proba_test.iloc[:, 1])

    pred4_proba_valid = model4.predict_proba(valid_gpu)
    pred4_valid = cp.asnumpy(pred4_proba_valid.iloc[:, 1])

    pred4_proba_test = model4.predict_proba(test_gpu)
    pred4_test = cp.asnumpy(pred4_proba_test.iloc[:, 1])

    # Store Level 1 predictions
    level1_train[valid_index, 0] = pred1_valid
    level1_train[valid_index, 1] = pred2_valid
    level1_train[valid_index, 2] = pred3_valid
    level1_train[valid_index, 3] = pred4_valid

    level1_test[:, 0] += pred1_test / FOLDS
    level1_test[:, 1] += pred2_test / FOLDS
    level1_test[:, 2] += pred3_test / FOLDS
    level1_test[:, 3] += pred4_test / FOLDS

    # Quick ensemble for this fold
    ensemble_pred = (pred1_valid + pred2_valid + pred3_valid + pred4_valid) / 4
    fold_auc = roc_auc_score(y_valid, ensemble_pred)
    print(f"Fold {fold + 1} Level 1 Ensemble AUC: {fold_auc:.4f}")

# LEVEL 2: Meta-learner
print("\n=== LEVEL 2: Training Meta-learner ===")
level1_gpu = cd.from_pandas(pd.DataFrame(level1_train))
y_gpu = cd.from_pandas(pd.Series(train_df['Personality_binary'].values))

meta_model = cuLogisticRegression(max_iter=2000)
meta_model.fit(level1_gpu, y_gpu)

level2_train_proba = meta_model.predict_proba(level1_gpu)
level2_train = cp.asnumpy(level2_train_proba.iloc[:, 1])

level2_test_proba = meta_model.predict_proba(cd.from_pandas(pd.DataFrame(level1_test)))
level2_test = cp.asnumpy(level2_test_proba.iloc[:, 1])

# LEVEL 3: Final blend
print("\n=== LEVEL 3: Final Blend ===")
# Simple weighted average (60% Level 2, 40% Level 1 ensemble)
level1_ensemble_train = np.mean(level1_train, axis=1)
level1_ensemble_test = np.mean(level1_test, axis=1)

final_train = 0.6 * level2_train + 0.4 * level1_ensemble_train
final_test = 0.6 * level2_test + 0.4 * level1_ensemble_test

# Final performance
final_auc = roc_auc_score(train_df['Personality_binary'], final_train)
final_f1 = f1_score(train_df['Personality_binary'], (final_train > 0.5).astype(int))
final_acc = accuracy_score(train_df['Personality_binary'], (final_train > 0.5).astype(int))

print(f"\n=== FINAL 3-LEVEL STACKING RESULTS ===")
print(f"AUC: {final_auc:.4f}")
print(f"F1: {final_f1:.4f}")
print(f"Accuracy: {final_acc:.4f}")

# Create submission
test_predictions_binary = (final_test > 0.5).astype(int)
test_predictions = np.where(test_predictions_binary == 1, 'Introvert', 'Extrovert')

submission_df = pd.DataFrame({
    'id': test_df['id'],
    'Personality': test_predictions
})

submission_df.to_csv('stacking_submission.csv', index=False)
print("3-Level Stacking submission saved!")

Starting 3-Level Stacking with Target Encoding...

=== FOLD 1 ===
Created 66 features
Training until validation scores don't improve for 50 rounds
Early stopping, best iteration is:
[546]	valid_0's binary_logloss: 0.136323
Fold 1 Level 1 Ensemble AUC: 0.9670

=== FOLD 2 ===
Created 66 features
Training until validation scores don't improve for 50 rounds
Early stopping, best iteration is:
[264]	valid_0's binary_logloss: 0.149383
Fold 2 Level 1 Ensemble AUC: 0.9665

=== FOLD 3 ===
Created 66 features
Training until validation scores don't improve for 50 rounds
Early stopping, best iteration is:
[201]	valid_0's binary_logloss: 0.151307
Fold 3 Level 1 Ensemble AUC: 0.9615

=== FOLD 4 ===
Created 66 features
Training until validation scores don't improve for 50 rounds
Early stopping, best iteration is:
[177]	valid_0's binary_logloss: 0.141988
Fold 4 Level 1 Ensemble AUC: 0.9661

=== FOLD 5 ===
Created 66 features
Training until validation scores don't improve for 50 rounds
Early stopping, b